In [1]:
import os
import pandas as pd
from nlpipe import NlPipe
import numpy as np
import os
from tqdm.auto import tqdm
import logging
from threadpoolctl import threadpool_limits

In [2]:
path = "b_collection_extracted/"

stat_df = pd.read_pickle(f"{path}stat_df")
if os.path.exists(f"{path}text_df"):
    print("text df found. loading.")
    text_df = pd.read_pickle(f"{path}text_df")
    texts = text_df.full_text.to_list()
    thread_ids = text_df.thread_id.to_list()
else:
    thread_ids = stat_df.thread_id.to_list()
    post_df = pd.read_pickle(f"{path}post_df_extracted")
    thread_id_of_posts = np.array(post_df.thread_id, dtype=np.uint32)
    texts = [" ".join(post_df.full_string[thread_id_of_posts == thread_id].tolist()) for thread_id in thread_ids]
    post_df = None
    text_df = pd.DataFrame([thread_ids, texts]).transpose()
    text_df.columns = ['thread_id', 'full_text']
    text_df.to_pickle(f"{path}text_df")

text df found. loading.


In [3]:
nlp = NlPipe.NlPipe(texts, path=path, document_ids=thread_ids, no_processes=10)
filter_array = np.logical_and(stat_df.language == 'en',
                              stat_df.replies >= 10)
print(f"{len(filter_array)} is limiting to {sum(filter_array)}")
nlp.preprocess(load_existing=True, filter_loaded=filter_array)
nlp.create_bag_of_words(filter_extremes=True, min_df=25, max_df=0.3)
with threadpool_limits(limits=1, user_api='blas'):
    nlp.create_lda_model(no_topics=20, passes=5, alpha='asymmetric', eta=0.01)
coherence_normal = nlp.calculate_coherence()
coherence_normal = coherence_normal.get_coherence()
nlp.create_mallet_lda_model(no_topics=20)
coherence_mallet = nlp.calculate_coherence()
coherence_mallet = coherence_mallet.get_coherence()
print(f"Coherence LDA-Multicore (Gensim): {coherence_normal}, coherence using MALLET:{coherence_mallet}")

326538 is limiting to 105583
Found preprocessed data. Loading
105583 preprocessed docs of 326538 docs loaded.


Creating bag of words:   0%|          | 0/105583 [00:00<?, ?it/s]

Coherence LDA-Multicore (Gensim): 0.41049825837304993, coherence using MALLET:0.5191123715619884
